In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [3]:
training = pd.DataFrame()
training = pd.read_csv("NCAA_Seed_Test_Set_2026_20260208.csv")
testing = pd.DataFrame()
testing = pd.read_csv("NCAA_Seed_Test_Set2.0.csv")

In [4]:
#num_neighbors = 50
ind_vars1 = ["Q1 Win", "Q1 Loss", "Q2 Win", "Q2 Loss", "Q3 Win", "Q3 Loss", "Q4 Win", "Q4 Loss"]
#ind_vars2 = ["NET Rank", "Q1 Win", "Q1 Loss", "Q2 Win", "Q2 Loss", "Q3 Win", "Q3 Loss", "Q4 Win", "Q4 Loss"]
#model = KNeighborsClassifier(n_neighbors = num_neighbors)

training_filt = training.dropna(subset = ["NET Rank"])
#testing_filt = testing.dropna(subset = ["NET Rank"])

#yTrain = training_filt['Legit Winner']
#yTest = testing_filt['Legit Winner']

#X1Train = training_filt[ind_vars1]
#X1Test = testing_filt[ind_vars1]

#X2Train = training_filt[ind_vars2]
#X2Test = testing_filt[ind_vars2]

In [ ]:
#model.fit(X = X1Train, y = yTrain)
#y_pred = model.predict(X1Test)

#print('Confusion Matrix:')
#print(metrics.confusion_matrix(y_true = yTest, y_pred = y_pred))
#print('*****************')
#print("Accuracy Train:", metrics.accuracy_score(y_true=yTrain, y_pred=model.predict(X1Train)))
#print("Accuracy Test:", metrics.accuracy_score(y_true=yTest, y_pred=y_pred))
#print('*****************')
#scores = cross_val_score(model, X1Train, yTrain, cv=5, scoring='accuracy')
#print("Cross-validated accuracy scores:", scores)
#print("Mean Cross-validated accuracy scores:", round(scores.mean(), 3))
#print("Standard deviation:", round(scores.std(), 3))

Confusion Matrix:
[[393   2]
 [  7  44]]
*****************
Accuracy Train: 0.9688427299703264
Accuracy Test: 0.9798206278026906
*****************
Cross-validated accuracy scores: [0.96296296 0.97037037 0.97037037 0.95539033 0.9739777 ]
Mean Cross-validated accuracy scores: 0.967
Standard deviation: 0.007


In [ ]:
#n_neighbors = [1, 5, 10, 20, 35, 50, 75, 100, 150, 200]
#for neighbor in n_neighbors:
    #model = KNeighborsClassifier(neighbor)
    #print('Number of Neighbors = ', neighbor)
    #model.fit(X1Train, yTrain)
    #y_pred_train = model.predict(X1Train)
    #print('Accuracy on training data = ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))
    #y_pred = model.predict(X1Test)
    #print('Accuracy on test data = ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))
    #print('*****************')

Number of Neighbors =  1
Accuracy on training data =  1.0
Accuracy on test data =  0.9730941704035875
*****************
Number of Neighbors =  5
Accuracy on training data =  0.9807121661721068
Accuracy on test data =  0.9753363228699552
*****************
Number of Neighbors =  10
Accuracy on training data =  0.9747774480712166
Accuracy on test data =  0.9775784753363229
*****************
Number of Neighbors =  20
Accuracy on training data =  0.9673590504451038
Accuracy on test data =  0.9775784753363229
*****************
Number of Neighbors =  35
Accuracy on training data =  0.9681008902077152
Accuracy on test data =  0.9753363228699552
*****************
Number of Neighbors =  50
Accuracy on training data =  0.9688427299703264
Accuracy on test data =  0.9798206278026906
*****************
Number of Neighbors =  75
Accuracy on training data =  0.9681008902077152
Accuracy on test data =  0.9775784753363229
*****************
Number of Neighbors =  100
Accuracy on training data =  0.9621661

In [5]:
# Get index of best NET Rank in each conference
best_idx = testing.groupby("Conference")["NET Rank"].idxmin()

# Assign AQ to those teams
testing.loc[best_idx, "Bid Type"] = "AQ"

In [ ]:
pred_df = testing.copy()
pred_df['Predicted_Legit_Winner'] = y_pred
pred_df.to_csv('knn_test_predictions.csv', index=False)

In [137]:
lr = linear_model.LinearRegression()
seed_train = training_filt.dropna(subset=["Bid Type"])
X_reg1 = seed_train[ind_vars1]
y_reg =seed_train["Overall Seed"]

XTrain, XTest, yTrain, yTest = train_test_split(X_reg1, y_reg, random_state=0, test_size=0.2)
lr.fit(X=XTrain, y=yTrain)

yPred = lr.predict(XTest)
print('r2: ', round(metrics.r2_score(yTest, yPred), 3))

scores = cross_val_score(lr, X_reg1, y_reg, cv=5, scoring='r2')

# Print results
print("Cross-validated R² scores:", scores)
print("Mean Cross-validated R² score:", round(scores.mean(), 3))
print("Standard deviation:", round(scores.std(), 3))

r2:  0.895
Cross-validated R² scores: [0.74260545 0.91998045 0.85332793 0.84169073 0.84040361]
Mean Cross-validated R² score: 0.84
Standard deviation: 0.057


In [ ]:
final_pred = testing
final_pred["Initial Seed"] = np.nan
filtered_pred = pred_df[
    (pred_df["Predicted_Legit_Winner"] == "Yes") |
    (pred_df["Bid Type"] == "AQ")
]
X_reg1_test = filtered_pred[ind_vars1]
predictions = lr.predict(X_reg1_test)
final_pred.loc[filtered_pred.index, "Initial Seed"] = predictions
final_pred["Overall Seed"] = final_pred["Initial Seed"].round(0)
final_pred["Overall Seed"] = final_pred["Overall Seed"].clip(lower=1, upper=68)
final_pred["Overall Seed"] = final_pred["Overall Seed"].fillna(0).astype(int)
final_pred = final_pred[["RecordID", "Overall Seed"]]
final_pred.to_csv('final_predictions.csv', index=False)